In [1]:
from typing import Optional
from llmp_agent.convert import convert_to_openai_tool, split_on_capital_case
from pydantic.v1 import BaseModel, validator
from datetime import datetime

from llmp_agent.tool_agent import BaseTool, OpenAIAgent, ToolResult


class AddExpense(BaseModel):
    description: str
    net_expense: float
    gross_expense: float
    tax_rate: float
    date: datetime
    
    @validator("date", pre=True)
    def parse_date(cls, v):
        if isinstance(v, str):
            for format in ["%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d"]:
                try:
                    return datetime.strptime(v, format)
                except:
                    pass
        return v
    
    
    
def add_expense(description: str, net_expense: float, gross_expense: float, tax_rate: float, date: datetime):
    if tax_rate > 0 and net_expense == gross_expense:
        return ToolResult(content="The net and gross expense are the same, but the tax rate is greater than 0. Please provide the correct net and gross expense.", success=False)
    if tax_rate > 0 and not gross_expense:
        return ToolResult(content="The tax rate is greater than 0, but the gross expense is not provided. Please provide the gross expense.", success=False)
    return f"Successfully added expense to database. Description: {description}, Net Expense: {net_expense}, Gross Expense: {gross_expense}, Tax Rate: {tax_rate}, Date: {date}"
    
class Report(BaseModel):
    result: str

def get_current_date(input: Optional[str] = None):
    return datetime.now().strftime("%Y-%m-%d")
    
db_tool = BaseTool.from_pydantic(AddExpense, add_expense)
report_tool = BaseTool.from_pydantic(Report)
date_tool = BaseTool.from_function(get_current_date)


In [2]:
date_tool.model

OutputModel(lines=[IOLine(key='input', type='Optional[str]', rule=None, options=None, multiple_select=False, default=None, placeholder=['input'], multiline=False, custom_value_template=None, hidden=False, description=None)])

In [3]:
date_tool.openai_tool

{'type': 'function',
 'function': {'name': 'get_current_date_tool',
  'description': '',
  'parameters': {'type': 'object',
   'properties': {'input': {'type': 'string'}},
   'required': []}}}

In [4]:
agent = OpenAIAgent(tools=[db_tool, report_tool, date_tool])
agent.run(input="I have spend 5.99 $ for a coffee today, please track my expense. The tax rate is 0.19.")

INFO: Running AgentOrchestor with {'input': 'I have spend 5.99 $ for a coffee today, please track my expense. The tax rate is 0.19.'}
Tool Call: Name: get_current_date_tool
Args: {}
Tool Result: content='2024-03-10' success=True metrics=None
Tool Call: Name: add_expense_tool
Args: {'description': 'Coffee purchase', 'net_expense': 5.99, 'gross_expense': 5.99, 'tax_rate': 0.19, 'date': '2024-03-10'}
Tool Result: content='The net and gross expense are the same, but the tax rate is greater than 0. Please provide the correct net and gross expense.' success=False metrics=None
Tool Call: Name: get_current_date_tool
Args: {}
Tool Result: content='2024-03-10' success=True metrics=None
Tool Call: Name: add_expense_tool
Args: {'description': 'Coffee purchase', 'net_expense': 5.99, 'gross_expense': 7.13, 'tax_rate': 0.19, 'date': '2024-03-10'}
Tool Result: content='Successfully added expense to database. Description: Coffee purchase, Net Expense: 5.99, Gross Expense: 7.13, Tax Rate: 0.19, Date: 20

'Expense for the coffee purchase has been successfully tracked.'

In [3]:
agent.token_usage

2236

In [3]:
from structgenie.components.validation import Validator

validator = Validator.from_pydantic(GetCurrentDate)

validator.validate({})

[]

In [3]:
validator.validation_config

{'input': {'type': 'str',
  'rule': None,
  'options': None,
  'multiple_select': False,
  'default': None,
  'multiline': False,
  'hidden': False,
  'description': None}}

In [2]:
from structgenie.components.input_output import OutputModel

output = OutputModel.from_pydantic(GetCurrentDate)
output

OutputModel(lines=[IOLine(key='input', type='Optional[str]', rule=None, options=None, multiple_select=False, default=None, placeholder=['input'], multiline=False, custom_value_template=None, hidden=False, description=None)])

In [7]:
GetCurrentDate.schema()

{'title': 'GetCurrentDate',
 'type': 'object',
 'properties': {'input': {'title': 'Input', 'type': 'string'}}}

In [22]:
class GetCurrentDate(BaseModel):
    input_x: Optional[str] = None
    
class GetCurrentDateOptional(BaseModel):
    input: str = None
    current: GetCurrentDate = None

GetCurrentDate.schema()

{'title': 'GetCurrentDate',
 'type': 'object',
 'properties': {'input_x': {'title': 'Input X', 'type': 'string'}}}

In [23]:
GetCurrentDateOptional.schema()

{'title': 'GetCurrentDateOptional',
 'type': 'object',
 'properties': {'input': {'title': 'Input', 'type': 'string'},
  'current': {'$ref': '#/definitions/GetCurrentDate'}},
 'definitions': {'GetCurrentDate': {'title': 'GetCurrentDate',
   'type': 'object',
   'properties': {'input_x': {'title': 'Input X', 'type': 'string'}}}}}

In [15]:
x = None

isinstance(x, Optional[str])

True

In [10]:
from colorama import Fore

Fore.__dict__

{'BLACK': '\x1b',
 'BLUE': '\x1b',
 'CYAN': '\x1b',
 'GREEN': '\x1b',
 'LIGHTBLACK_EX': '\x1b',
 'LIGHTBLUE_EX': '\x1b',
 'LIGHTCYAN_EX': '\x1b',
 'LIGHTGREEN_EX': '\x1b',
 'LIGHTMAGENTA_EX': '\x1b',
 'LIGHTRED_EX': '\x1b',
 'LIGHTWHITE_EX': '\x1b',
 'LIGHTYELLOW_EX': '\x1b',
 'MAGENTA': '\x1b',
 'RED': '\x1b',
 'RESET': '\x1b',
 'WHITE': '\x1b',
 'YELLOW': '\x1b'}

In [1]:
import inspect
from datetime import date
from typing import List
from structgenie.components.input_output.line import IOLine
from structgenie.components.input_output.output_model import OutputModel

def get_weather(location: str, date: date):
    pass  # function implementation

def create_output_model_from_function(func):
    sig = inspect.signature(func)
    lines = []
    for name, param in sig.parameters.items():
        if param.annotation is not param.empty:
            default = param.default if param.default is not param.empty else None
            line = IOLine(key=name, type=param.annotation.__name__, default=default)
            lines.append(line)
    return OutputModel(lines=lines)

output_model = create_output_model_from_function(get_weather)
print(output_model)

lines=[IOLine(key='location', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['location'], multiline=False, custom_value_template=None, hidden=False, description=None), IOLine(key='date', type='date', rule=None, options=None, multiple_select=False, default=None, placeholder=['date'], multiline=False, custom_value_template=None, hidden=False, description=None)]


In [23]:
import inspect
import datetime
from typing import List, get_args, Literal
from structgenie.components.input_output.line import IOLine
from structgenie.components.input_output.output_model import OutputModel

def get_weather(location: str = "San Francisco", date: datetime.date = None, weather_type: Literal['Sunny', 'Rainy', 'Cloudy'] = 'Sunny'):
    """Get the weather for a location and date.
    
    Args:
        location (str): The location to get the weather for.
        date (date): The date to get the weather for.
        weather_type (Literal['Sunny', 'Rainy', 'Cloudy']): The type of weather to get.
    """
    pass  # function implementation

def create_output_model_from_function(func):
    sig = inspect.signature(func)
    lines = []
    for name, param in sig.parameters.items():
        if param.annotation is not param.empty:
            default = param.default if param.default is not param.empty else None
            options = None
            type_ = param.annotation.__name__
            if getattr(param.annotation, '__origin__', None) == Literal:
                options = get_args(param.annotation)
                type_ = "string"
            line = IOLine(key=name, type=type_, default=default, options=options)
            lines.append(line)
    return OutputModel(lines=lines)

output_model = create_output_model_from_function(get_weather)
print(output_model)

lines=[IOLine(key='location', type='str', rule=None, options=None, multiple_select=False, default='San Francisco', placeholder=['location'], multiline=False, custom_value_template=None, hidden=False, description=None), IOLine(key='date', type='date', rule=None, options=None, multiple_select=False, default=None, placeholder=['date'], multiline=False, custom_value_template=None, hidden=False, description=None), IOLine(key='weather_type', type='str', rule=None, options=['Sunny', 'Rainy', 'Cloudy'], multiple_select=False, default='Sunny', placeholder=['weather_type'], multiline=False, custom_value_template=None, hidden=False, description=None)]


In [14]:
import inspect
import datetime
from typing import List, get_args, Literal
from structgenie.components.input_output.line import IOLine
from structgenie.components.input_output.output_model import OutputModel

def get_weather(location: Optional[str] = "San Francisco", date: datetime.date = None, weather_type: Literal['Sunny', 'Rainy', 'Cloudy'] = 'Sunny'):
    """Get the weather for a location and date.
    
    Args:
        location (str): The location to get the weather for.
        date (date): The date to get the weather for.
        weather_type (Literal['Sunny', 'Rainy', 'Cloudy']): The type of weather to get.
    """
    pass  # function implementation

def create_output_model_from_function(func):
    sig = inspect.signature(func)
    lines = []
    docstring = inspect.getdoc(func)
    doclines = docstring.split('\n')
    descriptions = {}
    for line in doclines:
        if ': ' in line:
            key, description = line.split(': ', 1)
            if "(" in key:
                key = key.split("(")[0]
            descriptions[key.strip()] = description.strip()

    for name, param in sig.parameters.items():
        if param.annotation is not param.empty:
            default = param.default if param.default is not param.empty else None
            options = None
            type_ = param.annotation.__name__
            print(type_)
            if getattr(param.annotation, '__origin__', None) == Literal:
                options = get_args(param.annotation)
                type_ = "string"
            description = descriptions.get(name, None)
            line = IOLine(key=name, type=type_, default=default, options=options, description=description)
            lines.append(line)
    return OutputModel(lines=lines)

output_model = create_output_model_from_function(get_weather)
print(output_model)

Optional
date
Literal
lines=[IOLine(key='location', type='Optional', rule=None, options=None, multiple_select=False, default='San Francisco', placeholder=['location'], multiline=False, custom_value_template=None, hidden=False, description='The location to get the weather for.'), IOLine(key='date', type='date', rule=None, options=None, multiple_select=False, default=None, placeholder=['date'], multiline=False, custom_value_template=None, hidden=False, description='The date to get the weather for.'), IOLine(key='weather_type', type='str', rule=None, options=['Sunny', 'Rainy', 'Cloudy'], multiple_select=False, default='Sunny', placeholder=['weather_type'], multiline=False, custom_value_template=None, hidden=False, description='The type of weather to get.')]


In [16]:
sig = inspect.signature(get_weather)
sig

<Signature (location: Optional[str] = 'San Francisco', date: datetime.date = None, weather_type: Literal['Sunny', 'Rainy', 'Cloudy'] = 'Sunny')>

In [19]:
for name, param in sig.parameters.items():
    print(name, param.annotation)
    print(param.annotation.__name__)
    

location typing.Optional[str]
Optional
date <class 'datetime.date'>
date
weather_type typing.Literal['Sunny', 'Rainy', 'Cloudy']
Literal


In [23]:
param = sig.parameters['weather_type']

param.annotation.__name__

'Literal'

In [18]:
def to_parameters_dict(self) -> dict:
    properties = {}
    for line in self.lines:
        properties[line.key] = {'type': line.type}
        if line.type == 'datetime':
            properties[line.key]['format'] = 'date-time'
            
    required = [line.key for line in self.lines if line.default is None or not "Optional" in line.type]
    parameters = {'type': 'object', 'properties': properties, "required": required}
    return parameters


to_parameters_dict(output_model)



{'type': 'object',
 'properties': {'location': {'type': 'str'},
  'date': {'type': 'str'},
  'weather_type': {'type': 'str'}},
 'required': ['location', 'date', 'weather_type']}